# 1. Preparations

## Imports

In [109]:
# Standard Library Imports
import ast
import copy
import csv
import json
import math
import os
import re
import time
import warnings
import logging
import random
import collections
from collections import Counter
from typing import List, Tuple, Optional
from IPython.display import HTML, display
import math
import time
from unidecode import unidecode
from pathlib import Path
import subprocess
from collections import Counter, defaultdict

# Data Handling Libraries
import numpy as np
import pandas as pd
import csv
from torch.utils.data import random_split
import datasets
from datasets import ClassLabel, Sequence

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt  # Uncomment if scikit-plot is installed and needed

# Machine Learning: Model Preparation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Machine Learning: Models and Frameworks
import tensorflow as tf
import torch
import evaluate
import xgboost
import wandb
from xgboost import plot_importance  # Uncomment if xgboost importance plot is required


# NLP and Transformers
from transformers import (AdamW, AutoModelForSequenceClassification, AutoModelForQuestionAnswering,
                          AutoTokenizer, CamembertForSequenceClassification, DistilBertConfig,
                          DistilBertForSequenceClassification, DistilBertModel, EarlyStoppingCallback,
                          get_linear_schedule_with_warmup, RobertaForSequenceClassification, EvalPrediction,
                          Trainer, TrainerCallback, TrainingArguments, XLMRobertaForSequenceClassification,
                         DefaultDataCollator, BertForQuestionAnswering, DataCollatorWithPadding, PreTrainedTokenizerFast,
                         default_data_collator, is_torch_xla_available)
from datasets import Dataset, DatasetDict, load_dataset
from transformers.trainer_utils import PredictionOutput, speed_metrics

# Experiment Tracking and Optimization Utilities
import optuna
from optuna.trial import TrialState
# import wandb  # Uncomment if using Weights & Biases for experiment tracking

# Progress Bar Utilities
from tqdm.notebook import tqdm


## Global Variables

In [4]:
## Arguments and global vriables
global_doc_stride = 128
data_collator = DefaultDataCollator()
version_2_with_negative = True
no_answer_threshold = 0.5
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)
global train_dataset
global eval_dataset
global cleaned_training_dataset

## Reusable Functions

In [72]:
# Function to generate predictions and format them
def generate_and_save_predictions(trainer, dataset, examples, filename="predictions.json"):
    # Predicting using the trainer
    predictions_output = trainer.predict(predict_dataset=dataset, predict_examples=examples)
    
    # Transform the list of predictions to the desired format
    formatted_predictions = {pred['id']: pred['prediction_text'] for pred in predictions_output.predictions}
        
    # Saving the predictions in the required format
    with open(filename, "w") as writer:
        json.dump(formatted_predictions, writer, indent=4)
        
    return predictions_output


In [6]:
### Compute_metrics function for Question and Answering problem is different to classification, more preocessing required.
metric = evaluate.load("squad_v2")

def compute_metrics(p: EvalPrediction):
        return metric.compute(predictions=p.predictions, references=p.label_ids)

In [7]:
"""
A subclass of `Trainer` specific to Question-Answering tasks
"""

if is_torch_xla_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            # Only the main node write the results by default
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics

        if self.args.should_log:
            # Only the main node log the results by default
            self.log(metrics)

        if self.args.tpu_metrics_debug or self.args.debug:
            # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

In [8]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if right_padding else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [9]:
# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=version_2_with_negative, #If true, some of the examples do not have an answer.
        n_best_size=20, #The total number of n-best predictions to generate when looking for an answer.
        max_answer_length=30, #The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        null_score_diff_threshold=no_answer_threshold, #The threshold used to select the null answer: if the best answer has a score that is less than the score of the null answer minus this threshold, the null answer is selected for this example. Only useful when `version_2_with_negative=True`.
        output_dir= None,
        log_level=logging.WARNING,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if version_2_with_negative:
        formatted_predictions = [
                {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
    else:
        formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]
    
    
    references = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [66]:
def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        if not os.path.isdir(output_dir):
            raise EnvironmentError(f"{output_dir} is not a directory.")

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [11]:
# List of questions to debug
debug_questions = [
    "What ideology was sponsored at the Ming court?",
    "Who stopped their trips to Ming China?",
    "Another question of interest"
]

# Training preprocessing
def preprocess_function(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]        
        question = examples["question"][sample_index]
        example_id = examples["id"][sample_index]
        
        # If no answers are given, set the cls_index as answer.
        if not answers or ('answer_start' not in answers and 'text' not in answers):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            print(answers)
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if right_padding else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if right_padding else 0):
                token_end_index -= 1
            
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Increase the end index here to make sure it includes the last character
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                safe_start_index = max(0, token_start_index - 1)
                tokenized_examples["start_positions"].append(safe_start_index)

                # Adjust this line to ensure that the end character is included
                # Decrement token_end_index to point exactly one past the end of the answer text
                while token_end_index >= 0 and offsets[token_end_index][1] > end_char:
                    token_end_index -= 1
                safe_end_index = min(len(input_ids) - 1, token_end_index + 1)
                tokenized_examples["end_positions"].append(safe_end_index)  # Adjust to +2 if needed
                
        # Debugging code
        global global_counter
        global traing_answer_mismatches
        #if question in debug_questions:
        if len(answers["answer_start"]) > 0:
            # Ensure indices are within bounds
            #safe_start_index = max(0, token_start_index - 1)
            #safe_end_index = min(len(input_ids) - 1, token_end_index + 1)

            # Decode the answer using safe indices
            decoded_answer = tokenizer.decode(input_ids[safe_start_index:safe_end_index])
            actual_answer = answers["text"][0] if answers["text"] else "No answer provided"
            normalized_actual_answer = actual_answer.lower().replace(" ", "")
            normalized_decoded_answer = decoded_answer.lower().replace(" ", "")
            if normalized_decoded_answer != normalized_actual_answer:
                if global_counter == 0: print('Mismatch Found')
                global_counter += 1 
                traing_answer_mismatches.append({
                    'ID': example_id,
                    'Question': question,
                    'Offsets': offsets,
                    'Input IDs':input_ids,
                    'Tokenized Text': tokenizer.decode(input_ids),
                    'Char start/end index': f"{start_char} / {end_char}",
                    'Token start/end index':f"{token_start_index} / {token_end_index}",
                    'Decoded Answer': decoded_answer,
                    'Actual Answer': actual_answer,                    
                })    
    return tokenized_examples

## Data Load

In [14]:
import json
from pathlib import Path

def load_squad_data(path):
    with open(path, 'rb') as file:
        squad_dict = json.load(file)
    return squad_dict

def extract_data(squad_dict):   
    # Initialize lists to store the flattened data components
    ids, titles, contexts, questions, answers = [], [], [], [], []

    # Iterate through the data and flatten the structure
    for article in squad_dict['data']:
        title = article['title']
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question_id = qa['id']
                question = qa['question']
                # Ensure each answer contains 'text' and 'answer_start' keys
                if 'answers' in qa and qa['answers']:
                    answer_texts = [answer['text'] for answer in qa['answers']]
                    answer_starts = [answer['answer_start'] for answer in qa['answers']]
                else:
                    # Handle the case where no answers are provided (as in SQuAD 2.0 for unanswerable questions)
                    answer_texts = [""]
                    answer_starts = [-1]

                # Append each component to the corresponding list
                ids.append(question_id)
                titles.append(title)
                contexts.append(context)
                questions.append(question)
                # Store answers as a dictionary
                answers.append({'text': answer_texts, 'answer_start': answer_starts})
                    
    return {
        'id': ids,
        'title': titles,
        'context': contexts,
        'question': questions,
        'answers': answers
    }

val_path = Path('Datasets/dev-v2.0.json')
val_squad_dict = load_squad_data(val_path)
flattened_data = extract_data(val_squad_dict)

# Create a Dataset from the flattened data
eval_examples = Dataset.from_dict(flattened_data)

# This dataset can now be used directly in your processing pipeline
print(eval_examples[0])

{'id': '56ddde6b9a695914005b9628', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'In what country is Normandy located?', 'answers': {'answer_start': [159, 159, 159, 159], 'text': ['France', 'France', 'France', 'France']}}


In [15]:
print(eval_examples.column_names)


['id', 'title', 'context', 'question', 'answers']


In [16]:
len(eval_examples)

11873

# 2. BERT Model Evaluation

In [32]:
path = "./best_models/bert-base-cased-best_model/checkpoint-8130"
tokenizer = AutoTokenizer.from_pretrained(path)
max_seq_length = 512
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

In [40]:
eval_dataset = eval_examples.map(
                prepare_validation_features,
                batched=True,
                remove_columns=eval_examples.column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )


Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [74]:
BERT_model = AutoModelForQuestionAnswering.from_pretrained(path)

BERT_training_args = TrainingArguments(
    output_dir=f"./Results",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,    
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)


BERT_trainer = QuestionAnsweringTrainer(
    model=BERT_model,
    args=BERT_training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)


In [68]:


# Generating predictions for evaluation dataset
BERT_outcome = generate_and_save_predictions(BERT_trainer,eval_dataset, eval_examples, "BERT_validation_predictions.json")


  0%|          | 0/11873 [00:00<?, ?it/s]

In [69]:
# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'BERT_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

{
  "exact": 57.78657458098206,
  "f1": 71.97695382840446,
  "total": 11873,
  "HasAns_exact": 39.27125506072875,
  "HasAns_f1": 67.69270796299412,
  "HasAns_total": 5928,
  "NoAns_exact": 76.24894869638352,
  "NoAns_f1": 76.24894869638352,
  "NoAns_total": 5945
}



# 3. DistilBERT Model Evaluation

In [135]:
path = "./best_models/distilbert-base-cased-best_model/checkpoint-4065"
tokenizer = AutoTokenizer.from_pretrained(path)
max_seq_length = 512
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

eval_dataset = eval_examples.map(
                prepare_validation_features,
                batched=True,
                remove_columns=eval_examples.column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )

DistilBERT_model = AutoModelForQuestionAnswering.from_pretrained(path)

DistilBERT_training_args = TrainingArguments(
    output_dir=f"./Results",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,    
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)


DistilBERT_trainer = QuestionAnsweringTrainer(
    model=BERT_model,
    args=BERT_training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Generating predictions for evaluation dataset
DistilBERT_outcome = generate_and_save_predictions(BERT_trainer,eval_dataset, eval_examples, "DistilBERT_validation_predictions.json")

# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'DistilBERT_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

{
  "exact": 0.22740672113198013,
  "f1": 3.2841753268791583,
  "total": 11873,
  "HasAns_exact": 0.11808367071524967,
  "HasAns_f1": 6.240386919034458,
  "HasAns_total": 5928,
  "NoAns_exact": 0.33641715727502103,
  "NoAns_f1": 0.33641715727502103,
  "NoAns_total": 5945
}



# 4. ALBERT Model Evaluation

In [88]:
path = "./best_models/albert-albert-base-v2-best_model/checkpoint-24081"
tokenizer = AutoTokenizer.from_pretrained(path)
max_seq_length = 512
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

eval_dataset = eval_examples.map(
                prepare_validation_features,
                batched=True,
                remove_columns=eval_examples.column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )

ALBERT_model = AutoModelForQuestionAnswering.from_pretrained(path)

ALBERT_training_args = TrainingArguments(
    output_dir=f"./Results",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,    
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)


ALBERT_trainer = QuestionAnsweringTrainer(
    model=ALBERT_model,
    args=ALBERT_training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Generating predictions for evaluation dataset
ALBERT_outcome = generate_and_save_predictions(ALBERT_trainer,eval_dataset, eval_examples, "ALBERT_validation_predictions.json")

# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'ALBERT_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

{
  "exact": 60.75128442685084,
  "f1": 76.68877003831479,
  "total": 11873,
  "HasAns_exact": 42.24021592442645,
  "HasAns_f1": 74.16089181256798,
  "HasAns_total": 5928,
  "NoAns_exact": 79.2094196804037,
  "NoAns_f1": 79.2094196804037,
  "NoAns_total": 5945
}



# 5. RoBERTa Model Evaluation

In [94]:
path = "./best_models/FacebookAI-roberta-base-finetuned-manual/checkpoint-24192"
tokenizer = AutoTokenizer.from_pretrained(path)
max_seq_length = 512
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

eval_dataset = eval_examples.map(
                prepare_validation_features,
                batched=True,
                remove_columns=eval_examples.column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )

RoBERTa_model = AutoModelForQuestionAnswering.from_pretrained(path)

RoBERTa_training_args = TrainingArguments(
    output_dir=f"./Results",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,    
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)


RoBERTa_trainer = QuestionAnsweringTrainer(
    model=RoBERTa_model,
    args=RoBERTa_training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Generating predictions for evaluation dataset
RoBERTa_outcome = generate_and_save_predictions(RoBERTa_trainer,eval_dataset, eval_examples, "RoBERTa_validation_predictions.json")

# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'RoBERTa_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

{
  "exact": 61.686178724837866,
  "f1": 78.09128924527415,
  "total": 11873,
  "HasAns_exact": 43.10053981106613,
  "HasAns_f1": 75.95780654675018,
  "HasAns_total": 5928,
  "NoAns_exact": 80.21867115222877,
  "NoAns_f1": 80.21867115222877,
  "NoAns_total": 5945
}



# 6. DeBERTa Model Evaluation

In [105]:
path = "./best_models/microsoft-deberta-v3-base-best_model/checkpoint-48780"
tokenizer = AutoTokenizer.from_pretrained(path)
max_seq_length = 512
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

eval_dataset = eval_examples.map(
                prepare_validation_features,
                batched=True,
                remove_columns=eval_examples.column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )

DeBERTa_model = AutoModelForQuestionAnswering.from_pretrained(path)

DeBERTa_training_args = TrainingArguments(
    output_dir=f"./Results",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,    
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)


DeBERTa_trainer = QuestionAnsweringTrainer(
    model=DeBERTa_model,
    args=DeBERTa_training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Generating predictions for evaluation dataset
DeBERTa_outcome = generate_and_save_predictions(DeBERTa_trainer,eval_dataset, eval_examples, "DeBERTa_validation_predictions.json")

# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'DeBERTa_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

{
  "exact": 67.11025014739324,
  "f1": 83.71077041879414,
  "total": 11873,
  "HasAns_exact": 47.25033738191633,
  "HasAns_f1": 80.4989840051194,
  "HasAns_total": 5928,
  "NoAns_exact": 86.91337258200168,
  "NoAns_f1": 86.91337258200168,
  "NoAns_total": 5945
}



# 7. Ensemble Evaluation

In [125]:
def ensemble_predictions(*args):
    # Combine all predictions into a single dictionary keyed by question ID
    all_predictions = defaultdict(list)
    for predictions in args:
        for q_id, answer in predictions.items():
            all_predictions[q_id].append(answer)
    
    # Choose the most common answer for each question
    final_predictions = {}
    for q_id, answers in all_predictions.items():
        answer_counter = Counter(answers)
        final_answer, _ = answer_counter.most_common(1)[0]
        final_predictions[q_id] = final_answer

    return final_predictions

# Load predictions from files
def load_predictions(filename):
    with open(filename, "r") as file:
        return json.load(file)

# Example loading and combining predictions
bert_predictions = load_predictions("BERT_validation_predictions.json")
#distilbert_predictions = load_predictions("DistilBERT_validation_predictions.json")
albert_predictions = load_predictions("ALBERT_validation_predictions.json")
roberta_predictions = load_predictions("RoBERTa_validation_predictions.json")
deberta_predictions = load_predictions("DeBERTa_validation_predictions.json")

# Create an ensemble of the predictions
ensemble_predictions = ensemble_predictions(
    deberta_predictions,
    albert_predictions, 
    bert_predictions, 
    #distilbert_predictions, 
    roberta_predictions
    
)

# Optionally, save the ensemble predictions
with open("ensemble_validation_predictions.json", "w") as file:
    json.dump(ensemble_predictions, file, indent=4)


In [127]:
# Define the command as a list
command = ['python', 'evaluate-v2.0.py', 'dev-v2.0.json', 'ensemble_validation_predictions.json']

# Run the command
process = subprocess.run(command, text=True, capture_output=True)

# Print the output
print(process.stdout)

# Check if there were any errors
if process.stderr:
    print("Errors:")
    print(process.stderr)

{
  "exact": 66.47014233976249,
  "f1": 82.18739513799694,
  "total": 11873,
  "HasAns_exact": 44.888663967611336,
  "HasAns_f1": 76.36824265746273,
  "HasAns_total": 5928,
  "NoAns_exact": 87.98990748528175,
  "NoAns_f1": 87.98990748528175,
  "NoAns_total": 5945
}

